## PERIN

Preprocess the data from the stancer pipeline and bring it into canonical form so it can be used by the sentiment graph parser.

In [3]:
# IMPORTS
import pandas as pd
import json
import os

In [4]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/Perin_Preprocessing", exist_ok=False)

In [8]:
TRAIN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_val.csv"

TRAIN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_train.json"
TEST_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_test.json"
VAL_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_val.json"

# Full phrase or Head mode
FULL_PHRASE = False

In [9]:
# read in each dataframe
input_paths = [TRAIN_INPUT_PATH, TEST_INPUT_PATH, VAL_INPUT_PATH]
dataframes = []

for ip in input_paths:
    df = pd.read_csv(ip, index_col=0)
    dataframes.append(df)


In [20]:
# inspect a df
dataframes[1]

,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,arg1_head_start,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
doc_id,,,,,,,,,,,,,,,,,,,,,
3e3b8da4120aac44b6f13a5288d7147db78a0b6509b762683acb9c77e3939189,angesteckt,46,56,anstecken,Müller,0,6,N,Müller,0,...,Müller,0,6,N,Müller,0,6,con,"Predicate(type='con', args=(Head(sentence=13, ...",Müller hatte sich in Doha während der Club-WM ...
113462308d7b18700752102f3adbff444746f6fad2f68653a81c4467dbe26eeb,beendet,4,11,beenden,Nils Mani ( 29,12,26,N,Nils,12,...,seine Karriere,29,43,N,Karriere,35,43,con,"Predicate(type='con', args=(Head(sentence=3, t...",Nun beendet Nils Mani ( 29 ) seine Karriere .
a1ea2d97d25995d732be1af47194669268529c09e339a20c8f8913b7a934c769,kritisiert,119,129,kritisieren,Oppositionelle Medienschaffende,0,32,N,Oppositionelle,0,...,Nexta,69,74,N,Nexta,69,74,con,"Predicate(type='con', args=(Head(sentence=30, ...","Oppositionelle Medienschaffende , die im Land ..."
f8482107fecc6dcc0ffcc503e63f25a1ada6308ea4f84596ba87d94a23be694c,verzichtet,48,58,verzichten,-Star,17,22,N,-Star,17,...,Geschenke,78,87,N,Geschenke,78,87,con,"Predicate(type='con', args=(Head(sentence=7, t...",Der « Bad Moms » -Star sammelt Rabattmarken un...
d1841e59308a81956665c185d5f447907f1e5bb957301929d0a5d82f3c05ac47,attackieren,50,61,attackieren,Die Immunzellen,0,15,N,Immunzellen,4,...,die Krebszellen,34,49,N,Krebszellen,38,49,con,"Predicate(type='con', args=(Head(sentence=59, ...",Die Immunzellen könnten so länger die Krebszel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ebd150015b1f91fedfa983b47f1501bffb6dc5676069a28319d57ebedf22f0ab,gestohlen,62,71,stehlen,Ein Mann,5,13,N,Mann,9,...,ein Ambulanzfahrzeug,41,61,N,Ambulanzfahrzeug,45,61,pro,"Predicate(type='pro', args=(Head(sentence=2, t...",Oslo Ein Mann hat gestern Mittag in Oslo ein A...
3b6b68a3ae73069e63c1dfd02abfa39b0a9f86c2c8dcab6109b80ee9e47e2518,preist,3,9,preisen,Xi,0,2,N,Xi,0,...,das chinesische Herrschaftssystem,10,43,N,Herrschaftssystem,26,43,pro,"Predicate(type='pro', args=(Head(sentence=13, ...",Xi preist das chinesische Herrschaftssystem vi...
60a73c8c406d7d7468634180300f7e39ad352816abe999bb75dc336160f0ed36,pflegen,17,24,pflegen,Digitale Nomaden,0,16,N,Nomaden,9,...,einen minimalistischen Lebensstil,32,65,N,Lebensstil,55,65,pro,"Predicate(type='pro', args=(Head(sentence=15, ...",Digitale Nomaden pflegen häufig einen minimali...


In [21]:
dataframes[1].rel_type.value_counts()

neutral    13493
pro        13415
con        13349
Name: rel_type, dtype: int64

In [22]:
def transform_to_semeval_ssa_format(df):
    sent_list = []
    sentences = list(set(df.full_sentence_text.to_list()))
    for i, sentence in enumerate(sentences):
        opinions_list = []
        sub_df = df[df["full_sentence_text"] == sentence]
        for row in sub_df.itertuples():
            # skip neutral case, not needed here
            if row.rel_type == "neutral":
                continue

            if FULL_PHRASE:
                # only-NPs
                opinions_list.append({
                    "Source": [[row.arg1], [f"{row.arg1_start}:{row.arg1_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_start}:{row.arg2_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else None,
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })
            else:
                # non-NPs
                opinions_list.append({
                    "Source": [[row.arg1_head], [f"{row.arg1_head_start}:{row.arg1_head_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_head_start}:{row.arg2_head_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else None,
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })

        if len(opinions_list) != 0:
            sent_list.append({
                "sent_id": str(i),
                "text": sentence,
                "opinions": opinions_list
            })
        else:
            continue
    return sent_list

In [23]:
# test the function
data = transform_to_semeval_ssa_format(dataframes[1])

In [24]:
output_paths = [TRAIN_OUTPUT_PATH, TEST_OUTPUT_PATH, VAL_OUTPUT_PATH]

for op, df in zip(output_paths, dataframes):
    # call function for each... dev, test, train-json
    data = transform_to_semeval_ssa_format(df)
    # convert to JSON format.
    with open(op, mode="w") as f:
        json.dump(data, f, ensure_ascii=False)